Job Assignment Problem

In [3]:
import heapq
import copy

class Node:
    
    def __init__(self, x, y, assigned, parent):
        
        self.parent = parent
        self.pathCost = 0
        self.cost = 0
        self.workerID = x
        self.jobID = y
        self.assigned = copy.deepcopy(assigned)
        
        if y != -1:
            self.assigned[y] = True

    def __lt__(self, other):
        
        return self.cost < other.cost  # Compare nodes based on their cost

class CustomHeap:
    
    def __init__(self):
        self.heap = []

    def push(self, node):
        heapq.heappush(self.heap, node)

    def pop(self):
        
        if self.heap:
            return heapq.heappop(self.heap)
        return None

def new_node(x, y, assigned, parent):
    return Node(x, y, assigned, parent)

def calc_cost(cost_mat, x, y, assigned, N):
    
    cost = 0
    avail = [True] * N
    
    for i in range(x + 1, N):
        min_val, min_index = float('inf'), -1
        
        for j in range(N):
            if not assigned[j] and avail[j] and cost_mat[i][j] < min_val:
                min_index = j
                min_val = cost_mat[i][j]
        cost += min_val
        avail[min_index] = False
    
    return cost

def print_assig(min_node):
    
    if min_node.parent is None:
        return
    
    print_assig(min_node.parent)
    print(f"Assign Worker {chr(min_node.workerID + ord('A'))} to Job {min_node.jobID + 1}")

def find_min_cost(cost_mat, N):
    
    pq = CustomHeap()
    assigned = [False] * N
    root = new_node(-1, -1, assigned, None)
    root.pathCost = root.cost = 0
    root.workerID = -1
    pq.push(root)

    while True:
        min_node = pq.pop()
        i = min_node.workerID + 1
        
        if i == N:
            print_assig(min_node)
            return min_node.cost

        for j in range(N):
            
            if not min_node.assigned[j]:
                child = new_node(i, j, min_node.assigned, min_node)
                child.pathCost = min_node.pathCost + cost_mat[i][j]
                child.cost = child.pathCost + calc_cost(cost_mat, i, j, child.assigned, N)
                pq.push(child)

if __name__ == "__main__":
    
    print("Job Assignment Problem")
    num_workers = int(input("Enter num of workers: "))
    num_jobs = int(input("Enter num of jobs: "))
    
    if num_workers != num_jobs:
        print("Num of workers and jobs must be equal.")
    
    else:
        print("Enter cost mat row by row (space-separated values):")
        cost_mat = []
        
        for i in range(num_workers):
            row = list(map(int, input(f"Row {i + 1}: ").split()))
            cost_mat.append(row)

        print("\nCalculating Optimal Assignment...")
        optimal_cost = find_min_cost(cost_mat, num_workers)

        if optimal_cost is not None:
            print(f"\nOptimal Cost is: {optimal_cost}")
        else:
            print("\nNo optimal solution found.")


Job Assignment Problem
Enter num of workers: 4
Enter num of jobs: 4
Enter cost mat row by row (space-separated values):
Row 1: 9 2 7 8
Row 2: 6 4 3 7
Row 3: 5 8 1 8
Row 4: 7 6 9 4

Calculating Optimal Assignment...
Assign Worker A to Job 2
Assign Worker B to Job 1
Assign Worker C to Job 3
Assign Worker D to Job 4

Optimal Cost is: 13


In [ ]:
# Job Assignment Problem Using Branch and Bound Strategy

# Theory Explanation:
# The Job Assignment Problem is a classical combinatorial optimization problem. 
# Given a set of workers and jobs, where each worker can be assigned to a job 
# at a certain cost, the goal is to find an optimal assignment (minimizing total cost) 
# where each worker is assigned exactly one job, and vice versa.
#
# In this code, the Branch and Bound strategy is used to solve this assignment problem.
# This strategy breaks the problem into smaller subproblems (branching) and uses bounds 
# to eliminate parts of the search space that cannot provide a better solution than the current best (bounding).
# It efficiently prunes the search space to avoid exploring unpromising assignments.



# Step-by-step Explanation of the Code:
# Node Class: The Node class represents a state in the search tree:
# - workerID: The worker being considered in this state.
# - jobID: The job assigned to the worker in this state.
# - assigned: A list that tracks which jobs have been assigned to workers so far.
# - parent: The parent node in the search tree (used for backtracking).
# - pathCost: The total cost of the path to reach this node.
# - cost: An estimated cost to reach the goal from this node. 
#   This is the sum of pathCost and the lower bound for the remaining assignments.

# CustomHeap Class: A min-heap is used to select the node with the minimum cost (priority queue). 
# This ensures that the algorithm explores the least costly path first.

# calc_cost Function: This function calculates the lower bound (or estimate) of the remaining cost from a given node.
# - It computes the minimum possible cost for the unassigned jobs from the current worker onward.
# - This estimate is used for bounding the search tree, eliminating paths that cannot yield a better solution than the best known solution.
# Formula used:
# cost = sum(min(c[i, j]) for j ∉ assigned_jobs for i from x+1 to N)
# where c[i,j] is the cost of assigning worker i to job j.

# find_min_cost Function: This is the main function that implements the Branch and Bound strategy. 
# It performs the following steps:
# - Initializes a priority queue (min-heap) and pushes the root node with an initial cost of zero.
# - Iteratively pops the node with the lowest cost from the heap and considers assigning the next worker (based on the workerID).
# - For each unassigned job, a new child node is created, and its path cost is updated.
# - The lower bound is calculated for the child node using calc_cost.
# - The child node is pushed back into the heap.
# - The process continues until all workers are assigned jobs (when i == N), at which point the assignment is printed and the total cost is returned.

# The key insight is that, by expanding nodes in order of their estimated cost (including both pathCost and lower bound), 
# we explore the most promising solutions first and prune suboptimal ones.

# Backtracking and Printing Assignments: The function print_assig recursively prints the job assignments starting from 
# the final node (leaf) to the root.

# Mathematical Formulation:
# The Job Assignment Problem can be represented as:
# Objective: Minimize the total cost C of assigning workers to jobs.
# C = sum(c[i, j] for i=1 to N)
# where c[i, j] is the cost of assigning worker i to job j.

# Constraints:
# - Each worker is assigned exactly one job.
# - Each job is assigned exactly one worker.



# Branch and Bound Approach:
# - Branching: At each level, a worker is assigned to a job, creating a branch. 
#   The search tree grows as workers are assigned jobs, and we explore all possible assignments.
# - Bounding:
#   - For each node, the cost is calculated, and a lower bound is used to estimate the remaining cost of completing the assignment.
#   - Nodes that cannot lead to a better solution than the best known solution are pruned, reducing the search space.
#   - This ensures that the algorithm explores promising nodes first and avoids unnecessary exploration of suboptimal solutions.



# Time and Space Complexity:
# Time Complexity: The algorithm explores N! possible assignments in the worst case, as there are N workers and N jobs. 
# However, by using bounding, the algorithm prunes much of the search space, potentially reducing the number of nodes explored. 
# In the worst case, the time complexity is O(N!), but the bounding reduces it significantly in practice.

# Space Complexity: The space complexity is dominated by the size of the search tree and the priority queue (heap),
# which could hold up to N! nodes. Therefore, the space complexity is also O(N!) in the worst case.



# Applications of Job Assignment Code:
# The Job Assignment problem is widely applicable in various fields:
# - Manufacturing and Production: Assigning workers to tasks or machines to minimize production costs.
# - Logistics: Assigning delivery vehicles to destinations to minimize transportation costs.
# - Project Management: Assigning team members to projects or tasks with different costs and skill requirements.
# - Airline Crew Scheduling: Assigning airline crew members to flights while minimizing the cost of assignments.



# Exam Questions and Answers:

# 1. What is the time complexity of the Branch and Bound algorithm used in this code?
# Answer: The time complexity in the worst case is O(N!) due to the factorial growth of possible assignments. 
# However, the algorithm uses bounding to prune much of the search space, so the actual time may be much less than this in practice.

# 2. What is the purpose of the calc_cost function in this code?
# Answer: The calc_cost function calculates the lower bound (or estimate) of the remaining cost for a given node. 
# It estimates the minimal cost required to complete the assignments for unassigned workers and jobs, 
# helping to prune suboptimal branches of the search tree.

# 3. Explain the significance of the pathCost and cost attributes in the Node class.
# Answer: The pathCost represents the total cost incurred to reach a node, while the cost represents the total estimated cost, 
# including the pathCost and the lower bound of the remaining assignments. The cost is used for selecting the node to expand next in the Branch and Bound strategy.

# 4. What is the role of the priority queue (min-heap) in this algorithm?
# Answer: The priority queue ensures that the node with the least cost is expanded first, 
# which is crucial for the efficiency of the Branch and Bound algorithm. 
# It allows the algorithm to explore the most promising assignments first and prune suboptimal ones quickly.

# 5. Why is it necessary to check if the number of workers and jobs are equal in the code?
# Answer: In the Job Assignment Problem, the number of workers must equal the number of jobs to ensure that each worker is assigned exactly one job 
# and each job gets exactly one worker. If the numbers do not match, it would not be possible to create a valid assignment, and the problem would be unsolvable.


In [ ]:
# Explanation of the Job Assignment Code
# The code solves the Job Assignment Problem using a branch and bound technique
# with a custom heap-based priority queue to find the optimal assignment of workers to jobs
# such that the overall cost is minimized.

# Step-by-Step Explanation:

# 1. Node Class
# The Node class represents a state in the branch and bound tree.
# It stores:
# - workerID: The current worker being considered.
# - jobID: The job being assigned to the current worker.
# - assigned: A list of boolean values indicating which jobs are assigned.
# - parent: Reference to the parent node in the tree.
# - pathCost: The total cost from the root node to the current node.
# - cost: The lower bound cost for exploring the node (including path cost and the heuristic).

# class Node:
#     def __init__(self, x, y, assigned, parent):
#         self.parent = parent
#         self.pathCost = 0
#         self.cost = 0
#         self.workerID = x
#         self.jobID = y
#         self.assigned = copy.deepcopy(assigned)
        
#         if y != -1:
#             self.assigned[y] = True  # Mark the job as assigned

#     def __lt__(self, other):
#         return self.cost < other.cost  # Compare nodes based on their cost

# 2. CustomHeap Class
# This class implements a custom priority queue using Python’s heapq module.
# Nodes are pushed into the heap based on their cost, and the node with the lowest cost is popped first,
# enabling branch and bound search.

# class CustomHeap:
#     def __init__(self):
#         self.heap = []

#     def push(self, node):
#         heapq.heappush(self.heap, node)

#     def pop(self):
#         if self.heap:
#             return heapq.heappop(self.heap)
#         return None

# 3. new_node Function
# This function creates a new node by passing the worker index (x), job index (y), the assigned jobs list, 
# and the parent node. It's used to build the branch and bound tree.

# def new_node(x, y, assigned, parent):
#     return Node(x, y, assigned, parent)

# 4. calc_cost Function
# The calc_cost function calculates the lower bound cost (heuristic) for a node.
# It looks ahead to the remaining workers and jobs and selects the minimum cost for unassigned jobs.
# It computes the minimum possible cost for the remaining workers and jobs to assign.
# The method finds the minimum possible cost by iterating over all unassigned jobs for each worker.

# def calc_cost(cost_mat, x, y, assigned, N):
#     cost = 0
#     avail = [True] * N
    
#     for i in range(x + 1, N):
#         min_val, min_index = float('inf'), -1
        
#         for j in range(N):
#             if not assigned[j] and avail[j] and cost_mat[i][j] < min_val:
#                 min_index = j
#                 min_val = cost_mat[i][j]
#         cost += min_val
#         avail[min_index] = False
    
#     return cost

# 5. print_assig Function
# This recursive function prints the optimal assignment path by traversing the tree from the leaf node to the root node.

# def print_assig(min_node):
#     if min_node.parent is None:
#         return
#     print_assig(min_node.parent)
#     print(f"Assign Worker {chr(min_node.workerID + ord('A'))} to Job {min_node.jobID + 1}")

# 6. find_min_cost Function
# This is the main function that solves the Job Assignment Problem using branch and bound.
# The root node represents the starting point, with no worker assigned.
# In each iteration, the node with the lowest cost is popped from the priority queue,
# and the next worker-job assignments are explored.
# The cost of the node is computed as the sum of the path cost and the heuristic cost (lower bound),
# which is then used to decide which node to explore next.
# The function terminates once all workers have been assigned jobs, printing the assignment and the minimum cost.

# def find_min_cost(cost_mat, N):
#     pq = CustomHeap()
#     assigned = [False] * N
#     root = new_node(-1, -1, assigned, None)
#     root.pathCost = root.cost = 0
#     pq.push(root)

#     while True:
#         min_node = pq.pop()
#         i = min_node.workerID + 1
        
#         if i == N:
#             print_assig(min_node)
#             return min_node.cost

#         for j in range(N):
#             if not min_node.assigned[j]:
#                 child = new_node(i, j, min_node.assigned, min_node)
#                 child.pathCost = min_node.pathCost + cost_mat[i][j]
#                 child.cost = child.pathCost + calc_cost(cost_mat, i, j, child.assigned, N)
#                 pq.push(child)



# Branch and Bound Strategy (Greedy Approach)
# The algorithm uses a branch and bound strategy to solve this problem:

# 1. Branching:
# The problem is broken down by making decisions about assigning workers to jobs.
# Each node represents an assignment decision, and the tree branches out as workers are assigned to jobs.

# 2. Bounding:
# For each node, we compute a lower bound on the total cost by adding the cost so far (pathCost) 
# and the minimum possible cost of assigning the remaining jobs (calc_cost).
# The lower bound is used to prune the search tree.
# If the cost of a node exceeds the current best-known solution, it is discarded, 
# ensuring that we only explore potentially optimal solutions.

# Greedy Approach:
# The greedy strategy is implemented by always selecting the node with the minimum cost (cost), 
# which is a combination of the path cost and the lower bound heuristic.
# This ensures that the algorithm explores the least costly options first, potentially leading to the optimal solution faster.

# Mathematical Formulation:
# Cost of a Node:
# cost = pathCost + lower_bound_cost

# Lower Bound Cost (heuristic):
# lower_bound = Σ(min(cost[i][j]))  for i from 1 to N, where the summation is over the remaining unassigned jobs for each worker.



# Time and Space Complexity:
# Time Complexity:
# In the worst case, we need to explore all possible assignments of workers to jobs, which leads to O(N!) possible solutions (where N is the number of workers/jobs).
# Each node requires finding the minimum cost for unassigned jobs, which takes O(N) time.
# The total number of nodes processed is at most N!, so the overall time complexity is O(N! × N).

# Space Complexity:
# The space complexity is dominated by the storage of the priority queue and the recursive function calls.
# The priority queue stores up to N! nodes, and each node stores a list of assigned jobs,
# leading to a space complexity of O(N! × N).



# Application of Job Assignment Code:
# The Job Assignment Problem has several practical applications, including:
# - Workforce management: Assigning tasks to workers in an optimal way to minimize labor costs.
# - Manufacturing: Assigning machines to tasks in a factory setup.
# - Resource allocation: Optimally assigning resources to tasks to minimize costs or time, such as in project management.
# - Task scheduling in distributed computing systems: Minimizing the cost of allocating tasks to processors.



# Exam Questions:
# Question 1:
# What does the calc_cost function do, and why is it important in the branch and bound strategy?

# Answer: The calc_cost function calculates the lower bound for the remaining assignments after a partial assignment.
# It finds the minimum possible cost for unassigned jobs and workers, and this helps prune the search tree, ensuring that only potentially optimal nodes are explored.

# Question 2:
# What is the time complexity of this algorithm, and what is the reason for it?

# Answer: The time complexity is O(N! × N). The N! comes from the number of possible assignments of workers to jobs,
# and the N factor comes from calculating the minimum cost for each node (which involves iterating over all remaining jobs for each worker).

# Question 3:
# Explain how the greedy strategy is implemented in the algorithm.

# Answer: The greedy strategy is implemented by always selecting the node with the minimum cost (cost), 
# which is a combination of the path cost and the lower bound heuristic. This ensures that the algorithm explores the least costly options first, potentially leading to the optimal solution faster.

# Question 4:
# What happens if the number of workers is not equal to the number of jobs?

# Answer: If the number of workers is not equal to the number of jobs, the algorithm prints an error message and does not proceed,
# as the assignment is not feasible. The problem assumes that each worker is assigned exactly one job, and each job is assigned to exactly one worker.
